In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.utils.data as Data
import torch.nn.functional as F
dtype = torch.FloatTensor

In [2]:
sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.

embedding_size = 2
sequence_len = len(sentences[0])
num_class = len(set(labels))
batch_size = 3
vocab = ' '.join(sentences).split()
vocab = list(set(vocab))
vocab_size = len(vocab)
word2idx = {w:i for i,w in enumerate(vocab)}


In [3]:
def make_data(sentences,labels):
    inputs = []
    for sentence in sentences:
        inputs.append([word2idx[w] for w in sentence.split()])
    targets = []
    for label in labels:
        targets.append(label)
    return inputs,targets
input_batch, target_batch = make_data(sentences, labels)
input_batch, target_batch = torch.LongTensor(input_batch),torch.LongTensor(target_batch)
dataset = Data.TensorDataset(input_batch, target_batch)
loader = Data.DataLoader(dataset, batch_size, True)
    

In [4]:
print(input_batch.size())

torch.Size([6, 3])


In [5]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN,self).__init__()
        self.W = nn.Embedding(vocab_size,embedding_size)
        output_channel = 3
        self.conv = nn.Sequential(nn.Conv2d(1,output_channel,(2,embedding_size)),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2,1))
                                 )
        self.fc = nn.Linear(output_channel,num_class)
    def forward(self,X):
        '''
        X = [batch_size,sequence_length]
        '''
        batch_size = X.shape[0]
        embedding_X = self.W(X) #[batch_size,sequence_length,embedding_size]
        embedding_X = embedding_X.unsqueeze(1) #[batch_size,1,sequence_length,embedding_size]
        conv = self.conv(embedding_X) #[batch_size,output_channel,1,1]
        conv = conv.view(batch_size,-1)
        output = self.fc(conv)
        return output
        

In [6]:
model = TextCNN().cuda()
optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)
criterion  = nn.CrossEntropyLoss()
for epoch in range(5000):
    for batch_x,batch_y in loader:
        pred = model(batch_x.cuda())
        loss = criterion(pred,batch_y.cuda())
        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch: 1000 loss = 0.005023
Epoch: 1000 loss = 0.002371
Epoch: 2000 loss = 0.000423
Epoch: 2000 loss = 0.000376
Epoch: 3000 loss = 0.000051
Epoch: 3000 loss = 0.000130
Epoch: 4000 loss = 0.000012
Epoch: 4000 loss = 0.000038
Epoch: 5000 loss = 0.000008
Epoch: 5000 loss = 0.000007


In [17]:
# Test
test_text = 'likes is i'
tests = [[word2idx[n] for n in test_text.split()]]
test_batch = torch.LongTensor(tests).cuda()
# Predict
model = model.eval()
predict = model(test_batch).data.max(1, keepdim=True)[1]
if predict[0][0] == 0:
    print(test_text,"is Bad Mean...")
else:
    print(test_text,"is Good Mean!!")

likes is i is Bad Mean...
